In [152]:
import sys
sys.path.append('../main')
import sql
import pandas as pd
from functools import reduce
from tabulate import tabulate
pd.options.display.float_format = '{:,.1f}'.format

In [153]:
q='''select name, count(*) as total from alert a
        join project p on a.project_id = p.id
        join memory_error me on a.alert_type_id = me.alert_type_id
        where is_invalid=0
        group by name
        order by name'''
ta = pd.DataFrame(sql.execute(q)) 
q='''select name, count(*) as memory from alert a
        join project p on a.project_id = p.id
        join memory_error me on a.alert_type_id = me.alert_type_id
        where is_invalid=0
        and me.memory =1 
        group by name
        order by name'''
tma = pd.DataFrame(sql.execute(q)) 
tma['memory (%)']= round((tma.memory / ta.total) * 100,2)
q='''select name, count(*) as non_memory from alert a
        join project p on a.project_id = p.id
        join memory_error me on a.alert_type_id = me.alert_type_id
        where is_invalid=0
        and me.memory is null
        group by name
        order by name'''
tnma = pd.DataFrame(sql.execute(q)) 
tnma['non_memory (%)']= round((tnma.non_memory / ta.total) * 100,2)
df= reduce(lambda x,y: pd.merge(x,y, on='name', how='outer'),[ta,tma,tnma])   
df = df.sort_values('total',ascending=False)

In [154]:
df['memory (%)'].median()

37.72

In [155]:
df['memory']= df['memory'].astype(str)+' ('+df['memory (%)'].astype(str)+'%)'
df=df.drop(columns=['non_memory', 'non_memory (%)', 'memory (%)'])
df=df.rename(columns={'name':'Project','total':'Total Alerts', 'memory':'Memory Alerts'})
print(tabulate(df, tablefmt='pipe',showindex=False, headers='keys'))

| Project               |   Total Alerts | Memory Alerts   |
|:----------------------|---------------:|:----------------|
| Linux                 |          19514 | 7903 (40.5%)    |
| Firefox               |          12945 | 5448 (42.09%)   |
| LibreOffice           |          12515 | 4194 (33.51%)   |
| Samba                 |           4507 | 1700 (37.72%)   |
| ovirt-engine          |           2989 | 1272 (42.56%)   |
| VTK                   |           2522 | 1166 (46.23%)   |
| OpenCV                |           2443 | 710 (29.06%)    |
| Kodi                  |           2393 | 682 (28.5%)     |
| !CHAOS Control System |           1980 | 282 (14.24%)    |
| Chromium EC           |            964 | 306 (31.74%)    |
| Thunderbird           |            885 | 357 (40.34%)    |


In [156]:
q='''select * from alert a
join project p on a.project_id = p.id
join memory_error me on a.alert_type_id = me.alert_type_id
where is_invalid=0
and me.memory =1
order by name;'''
root=pd.DataFrame(sql.execute(q)) 
#root.head()
total=root.groupby('name')[['cid']].count()
total.rename(columns={'cid':'total'},inplace=True)
eliminated=root[root['status']=='Fixed'].groupby('name')[['cid']].count()
eliminated.rename(columns={'cid':'e'},inplace=True)
bug=root[root['classification']=='Bug'].groupby('name')[['cid']].count()
q='''select * from alert a
join project p on a.project_id = p.id
join memory_error me on a.alert_type_id = me.alert_type_id
join actionability ac on ac.alert_id=a.id
where is_invalid=0
and actionability = 1
and me.memory =1
order by name;'''
df=pd.DataFrame(sql.execute(q))
actionable=df.groupby('name')[['cid']].count()
actionable.rename(columns={'cid':'a'},inplace=True)
bug.rename(columns={'cid':'b'},inplace=True)
df=reduce(lambda x,y:pd.merge(x,y,on='name'),[total,eliminated, actionable, bug])
df.e = round((df.e/df.total)*100,2)    
df.a = round((df.a/df.total)*100,2)
df.b = round((df.b/df.total)*100,2)  
df=df.reset_index() 

In [157]:
# df.e=df.e.astype(str)+'%'
# df.a=df.a.astype(str)+'%'
# df.b=df.b.astype(str)+'%'
df.rename(columns={'name':'Project','total':'Total Memory Alerts', 'e':'Eliminated Memory', 'a':'Actionable Memory','b':'Triaged Bug by Devs. Memory'}, inplace=True)
memory = df 
memory

,Project,Total Memory Alerts,Eliminated Memory,Actionable Memory,Triaged Bug by Devs. Memory
0,!CHAOS Control System,282,80.1,39.0,1.4
1,Chromium EC,306,93.5,29.7,17.6
2,Firefox,5448,77.2,49.7,12.4
3,Kodi,682,68.6,54.1,25.1
4,LibreOffice,4194,80.6,73.8,52.4
5,Linux,7903,73.1,43.5,3.1
6,OpenCV,710,89.2,32.5,0.3
7,Samba,1700,77.9,39.9,3.8
8,Thunderbird,357,63.3,24.4,12.6
9,VTK,1166,12.8,7.2,0.4


In [158]:
       
memory = memory.sort_values('Total Memory Alerts', ascending=False)
print(tabulate(memory, tablefmt='pipe',headers='keys',showindex=False))

| Project               |   Total Memory Alerts |   Eliminated Memory |   Actionable Memory |   Triaged Bug by Devs. Memory |
|:----------------------|----------------------:|--------------------:|--------------------:|------------------------------:|
| Linux                 |                  7903 |               73.06 |               43.54 |                          3.11 |
| Firefox               |                  5448 |               77.22 |               49.72 |                         12.43 |
| LibreOffice           |                  4194 |               80.64 |               73.75 |                         52.43 |
| Samba                 |                  1700 |               77.88 |               39.88 |                          3.82 |
| ovirt-engine          |                  1272 |               36.08 |               27.12 |                          2.59 |
| VTK                   |                  1166 |               12.78 |                7.2  |                         

In [164]:
q='''select * from alert a
join project p on a.project_id = p.id
join memory_error me on a.alert_type_id = me.alert_type_id
where is_invalid=0
and me.memory is null
order by name;'''
root=pd.DataFrame(sql.execute(q)) 
total=root.groupby('name')[['cid']].count()
total.rename(columns={'cid':'total'},inplace=True)
eliminated=root[root['status']=='Fixed'].groupby('name')[['cid']].count()
eliminated.rename(columns={'cid':'e'},inplace=True)
bug=root[root['classification']=='Bug'].groupby('name')[['cid']].count()
q='''select * from alert a
join project p on a.project_id = p.id
join memory_error me on a.alert_type_id = me.alert_type_id
join actionability ac on ac.alert_id=a.id
where is_invalid=0
and actionability = 1
and me.memory is null
order by name;'''
df=pd.DataFrame(sql.execute(q))
actionable=df.groupby('name')[['cid']].count()
actionable.rename(columns={'cid':'a'},inplace=True)
bug.rename(columns={'cid':'b'},inplace=True)
df=reduce(lambda x,y:pd.merge(x,y,on='name'),[total,eliminated, actionable, bug])
df=df.reset_index()
df.e = round((df.e/df.total)*100,2)    
df.a = round((df.a/df.total)*100,2)
df.b = round((df.b/df.total)*100,2)   
# df.e=df.e.astype(str)+'%'
# df.a=df.a.astype(str)+'%'
# df.b=df.b.astype(str)+'%'
df.rename(columns={'name':'Project','total':'Total Non-Memory Alerts', 'e':'Eliminated Non-memory', 'a':'Actionable Non-memory','b':'Triaged Bugs by Devs. Non-memory'}, inplace=True)
non_memory = df        
#non_memory = non_memory.sort_values('total non_memory', ascending=False)
non_memory

,Project,Total Non-Memory Alerts,Eliminated Non-memory,Actionable Non-memory,Triaged Bugs by Devs. Non-memory
0,Chromium EC,658,71.6,28.0,8.5
1,Firefox,7497,70.9,46.5,5.1
2,Kodi,1711,67.3,50.3,11.6
3,LibreOffice,8321,94.1,71.0,41.4
4,Linux,11611,65.2,39.1,3.4
5,OpenCV,1733,76.5,36.0,0.4
6,Samba,2807,73.3,24.7,1.5
7,Thunderbird,528,49.8,15.0,5.1
8,VTK,1356,19.1,7.6,1.8
9,ovirt-engine,1717,47.8,32.6,2.6


In [165]:
df=reduce(lambda x,y: pd.merge(x,y,on='Project'),[memory,non_memory])
df = df.sort_values('Total Memory Alerts', ascending=False)
df

,Project,Total Memory Alerts,Eliminated Memory,Actionable Memory,Triaged Bug by Devs. Memory,Total Non-Memory Alerts,Eliminated Non-memory,Actionable Non-memory,Triaged Bugs by Devs. Non-memory
0,Linux,7903,73.1,43.5,3.1,11611,65.2,39.1,3.4
1,Firefox,5448,77.2,49.7,12.4,7497,70.9,46.5,5.1
2,LibreOffice,4194,80.6,73.8,52.4,8321,94.1,71.0,41.4
3,Samba,1700,77.9,39.9,3.8,2807,73.3,24.7,1.5
4,ovirt-engine,1272,36.1,27.1,2.6,1717,47.8,32.6,2.6
5,VTK,1166,12.8,7.2,0.4,1356,19.1,7.6,1.8
6,OpenCV,710,89.2,32.5,0.3,1733,76.5,36.0,0.4
7,Kodi,682,68.6,54.1,25.1,1711,67.3,50.3,11.6
8,Thunderbird,357,63.3,24.4,12.6,528,49.8,15.0,5.1
9,Chromium EC,306,93.5,29.7,17.6,658,71.6,28.0,8.5


In [170]:
from scipy.stats import shapiro
stat, p = shapiro(df['Eliminated Memory'])
for col in df.columns:
    if col=='Project':
        continue
    stat, p = shapiro(df[col])
    print(col, stat, p)

Total Memory Alerts 0.7904914021492004 0.011109322309494019
Eliminated Memory 0.8522984385490417 0.061861250549554825
Actionable Memory 0.9814115762710571 0.9722718596458435
Triaged Bug by Devs. Memory 0.7839808464050293 0.009256302379071712
Total Non-Memory Alerts 0.786896824836731 0.01004474051296711
Eliminated Non-memory 0.9119260311126709 0.29448455572128296
Actionable Non-memory 0.9802671670913696 0.9665817618370056
Triaged Bugs by Devs. Non-memory 0.6153701543807983 8.437768701696768e-05


In [171]:
from scipy.stats import mannwhitneyu
stat, p = mannwhitneyu(df['Eliminated Memory'],df['Eliminated Non-memory']
)
stat, p

(39.0, 0.21367765694890384)

In [172]:
from scipy.stats import mannwhitneyu
stat, p = mannwhitneyu(df['Actionable Memory'],df['Actionable Non-memory']
)
stat, p

(46.0, 0.3956683900503302)

In [175]:
from scipy.stats import mannwhitneyu
stat, p = mannwhitneyu(df['Triaged Bug by Devs. Memory'],df['Triaged Bugs by Devs. Non-memory']
)
stat, p

(40.0, 0.23633779675579358)

In [176]:
df['Eliminated Memory'].median()

75.14

In [177]:
df['Actionable Memory'].median()

36.21

In [179]:
df['Triaged Bug by Devs. Memory'].median()

8.125